In [1]:
import numpy as np
import pandas as pd
import sys
import os

from eppy import hvacbuilder
from eppy import modeleditor
from eppy.modeleditor import IDF

In [2]:
## this function determines what idf version and connects the idd for eppy

def eppy_connection(idffile):
    fp = open(idffile)
    lines=fp.readlines()
    vers = ""
    vers = vers.join(lines[2:12])
    vers
    if vers.find('8.7') != -1:
        print('Using EnergyPlus version 8.7')
        iddfile = 'C:\\EnergyPlusV8-7-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.8') != -1:
        print('Using EnergyPlus version 8.8')
        iddfile = 'C:\\EnergyPlusV8-8-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('8.9') != -1:
        print('Using EnergyPlus version 8.9')
        iddfile = 'C:\\EnergyPlusV8-9-0\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    elif vers.find('9.0') != -1:
        print('Using EnergyPlus version 9.0.1')
        iddfile = 'C:\\EnergyPlusV9-0-1\\Energy+.idd'
        IDF.setiddname(iddfile)
        idf = IDF(idffile)
        return idf
    else:
        print('IDF Type not found on line 3')

In [3]:
## CONNECT IDFS

#define paths
commonSlab_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\common_hp_slab_IECC_2012_V8.7.idf'
commonBsmt_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\common_hp_heatedbsmtIECC_2012_V8.7.idf'
gardenSlab_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\garden_hp_slab_IECC_2012_V8.7.idf'
gardenBsmt_path = 'C:\\Users\\scott\\github\\DOE_LRMF\\SEED_Models\\garden_hp_heatedbsmtIECC_2012_V8.7.idf'

#make connections
commonSlab = eppy_connection(commonSlab_path)
commonBsmt = eppy_connection(commonBsmt_path)
gardenSlab = eppy_connection(gardenSlab_path)
gardenBsmt = eppy_connection(gardenBsmt_path)

# create groups
idfs = [commonSlab, commonBsmt, gardenSlab, gardenBsmt]
idfs_strings = ['commonSlab', 'commonBsmt', 'gardenSlab', 'gardenBsmt']
common = [commonSlab, commonBsmt]
common_strings = ['commonSlab', 'commonBsmt']
garden = [gardenSlab, gardenBsmt]
garden_strings = ['gardenSlab', 'gardenBsmt']
bsmt = [commonBsmt, gardenBsmt]
bsmt_strings = ['commonBsmt', 'gardenBsmt']
slab = [commonSlab, gardenSlab]
slab_strings = ['commonSlab', 'gardenSlab']

# matching dictionaries
model_dict = {'idfs':idfs, 'common':common, 'garden':garden, 'bsmt':bsmt, 'slab':slab}
model_dict_strings = {'idfs':idfs_strings, 'common':common_strings, 'garden':garden_strings,
                      'bsmt':bsmt_strings, 'slab':slab_strings}
model_match = {'commonSlab':commonSlab,'commonBsmt':commonBsmt,
               'gardenSlab':gardenSlab,'gardenBsmt':gardenBsmt}

Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7
Using EnergyPlus version 8.7


In [4]:
## IMPORT INPUT DATAFRAMES

path = 'C:\\Users\\scott\\github\\eppy_scripts\\DOE_LRMF\\inputs\\'
individual_vars_filename = 'individual_vars.csv'
dictionary_vars_filename = 'dictionary_vars.csv'

individual_vars = pd.read_csv(path + individual_vars_filename)
dictionary_vars = pd.read_csv(path + dictionary_vars_filename)

In [5]:
change_log_detail=open('change_log_detail.txt','w')
change_log=open('change_log.txt','w')

### Create Dictionaries

#### Heating and Cooling DX

In [6]:
## Corridor

# create empty dictionary
corridorsDXheat_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(common_strings)):
    
    objs = common[i].idfobjects['Coil:Heating:DX:SingleSpeed']
    objs = [x for x in objs if 'Corridor' in x.Name]
    
    corridorsDXheat_dict.update({common_strings[i]: objs})
    
# create empty dictionary
corridorsDXcool_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(common_strings)):
    
    objs = common[i].idfobjects['Coil:Cooling:DX:SingleSpeed']
    objs = [x for x in objs if 'Corridor' in x.Name]
    
    corridorsDXcool_dict.update({common_strings[i]: objs})

In [7]:
## Basement

# create empty dictionary
bsmtDXheat_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(bsmt_strings)):
    
    objs = bsmt[i].idfobjects['Coil:Heating:DX:SingleSpeed']
    objs = [x for x in objs if 'unitheatedbsmt' in x.Name]
    
    bsmtDXheat_dict.update({bsmt_strings[i]: objs})
    
# create empty dictionary
bsmtDXcool_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(bsmt_strings)):
    
    objs = bsmt[i].idfobjects['Coil:Cooling:DX:SingleSpeed']
    objs = [x for x in objs if 'unitheatedbsmt' in x.Name]
    
    bsmtDXcool_dict.update({bsmt_strings[i]: objs})

In [8]:
## Units

# create empty dictionary
unitDXheat_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(idfs_strings)):
    
    objs = idfs[i].idfobjects['Coil:Heating:DX:SingleSpeed']
    objs1 = [x for x in objs if 'BackRow' in x.Name]
    objs2 = [x for x in objs if 'FrontRow' in x.Name]
    objs = objs1 + objs2
    
    unitDXheat_dict.update({idfs_strings[i]: objs})
    
# create empty dictionary
unitDXcool_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(idfs_strings)):
    
    objs = idfs[i].idfobjects['Coil:Cooling:DX:SingleSpeed']
    objs1 = [x for x in objs if 'BackRow' in x.Name]
    objs2 = [x for x in objs if 'FrontRow' in x.Name]
    objs = objs1 + objs2
    
    unitDXcool_dict.update({idfs_strings[i]: objs})

In [9]:
## Water Heaters

# create empty dictionary
waterHeaters_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(idfs_strings)):
    
    objs = idfs[i].idfobjects['WaterHeater:Mixed']
    waterHeaters_dict.update({idfs_strings[i]: objs})

In [10]:
## Corridor Lights

# create empty dictionary
corridorLights_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(common_strings)):
    
    objs = common[i].idfobjects['Lights']
    objs = [x for x in objs if 'Corridor' in x.Name]
    
    corridorLights_dict.update({common_strings[i]: objs})

In [11]:
## Unit Lights

# create empty dictionary
unitLights_dict = {}

# loop through idfs and create dictionary
for i in range(0, len(idfs_strings)):
    
    objs = idfs[i].idfobjects['Lights']
    objs1 = [x for x in objs if 'BackRow' in x.Name]
    objs2 = [x for x in objs if 'FrontRow' in x.Name]
    objs = objs1 + objs2
    
    unitLights_dict.update({idfs_strings[i]: objs})

In [12]:
# create matching dictionary for text references

matching_dict = {'CorridorsDXheat_dict':corridorsDXheat_dict,'CorridorsDXcool_dict':corridorsDXcool_dict,
                 'BsmtDXheat_dict':bsmtDXheat_dict,'BsmtDXcool_dict':bsmtDXcool_dict,
                 'UnitDXheat_dict':unitDXheat_dict,'UnitDXcool_dict':unitDXcool_dict,
                 'WaterHeaters_dict':waterHeaters_dict,'CorridorLights_dict':corridorLights_dict,
                 'UnitLights_dict':unitLights_dict}

## Modify IDFs for Individual Objects

In [13]:
change_log.write('#### UPDATES FOR INDIVIDUAL OBJECTS ####')
change_log.write('\n')

change_log_detail.write('#### UPDATES FOR INDIVIDUAL OBJECTS ####')
change_log_detail.write('\n')

# for each row
for i in range(0,len(individual_vars)):
    var = individual_vars['jEplus - Variable'][i]
    obj_type = individual_vars['Object Type'][i]
    obj_name = individual_vars['Name'][i]
    value = individual_vars['Value'][i]
    model_key = individual_vars['Models'][i]
    
    # get models and model strings from model key
    models = model_dict[model_key]
    model_strings = model_dict_strings[model_key]

    # log row update
    change_log.write('\n')
    change_log.write('Change '+obj_type+' '+obj_name+' '+value+' to '+var+' for '+model_key)
    
    # write to change log
    change_log_detail.write('\n')
    change_log_detail.write('\n')
    change_log_detail.write('Change '+obj_type+' '+obj_name+' '+value+' to '+var+' for '+model_key)
    
    # for each model
    for i in range(0, len(models)):

        # log model update
        change_log.write('\n')
        change_log.write('\tChange '+obj_type+' '+obj_name+' '+value+' to '+var+' for '+model_strings[i])
        
        # write to change log
        change_log_detail.write('\n')
        change_log_detail.write('\n')
        
        # find object in idf
        objs = models[i].idfobjects[obj_type]
        obj = [x for x in objs if x.Name == obj_name][0]

        # write to change log
        change_log_detail.write('Starting object')
        change_log_detail.write('\n')
        change_log_detail.write(obj.__str__())

        # change value
        setattr(obj, value, var)

        # write to change log
        change_log_detail.write('Modified object')
        change_log_detail.write('\n')
        change_log_detail.write(obj.__str__())

## Modify IDFs for Dictionary Objects

In [14]:
change_log.write('\n')
change_log.write('\n')
change_log.write('#### UPDATES FOR DICTIONARY OBJECTS ####')
change_log.write('\n')
change_log_detail.write('\n')
change_log_detail.write('\n')
change_log_detail.write('#### UPDATES FOR INDIVIDUAL OBJECTS ####')
change_log_detail.write('\n')

# for each row
for i in range(0,len(dictionary_vars)):
    
    var = dictionary_vars['jEplus - Variable'][i]
    obj_type = dictionary_vars['Object Type'][i]
    obj_dict = dictionary_vars['Name'][i]
    value = dictionary_vars['Value'][i]
    model_key = dictionary_vars['Models'][i]

    # log row update
    change_log.write('\n')
    change_log.write('Change '+obj_type+' '+obj_dict+' '+value+' to '+var+' for '+model_key)
    # write to change log
    change_log_detail.write('\n')
    change_log_detail.write('\n')
    change_log_detail.write('Change '+obj_type+' '+obj_dict+' '+value+' to '+var+' for '+model_key)

    # match obj_name to dictionary created above
    obj_names_dict = matching_dict[obj_dict]    
    
    # for each idf
    for key in obj_names_dict:
        
        # log model update
        change_log.write('\n')
        change_log.write('\tChange '+obj_type+' '+obj_name+' '+value+' to '+var+' for '+key)
        
        # obj list
        obj_list = obj_names_dict[key]
        # pull idf
        model = model_match[key]
        
        # for each object
        for i in range(0, len(obj_list)):

            # find object in idf
            obj_name = obj_list[i].Name
            objs = model.idfobjects[obj_type]
            obj = [x for x in objs if x.Name == obj_name][0]

            # write to change log
            change_log.write('\n')
            change_log.write('\t\tChange '+obj_type+' '+obj_name+' '+value+' to '+var+' for '+key)
            
            change_log_detail.write('\n')
            change_log_detail.write('\n')
            change_log_detail.write('Starting object')
            change_log_detail.write('\n')
            change_log_detail.write(obj.__str__())

            # change value
            setattr(obj, value, var)

            # write to change log
            change_log_detail.write('Modified object')
            change_log_detail.write('\n')
            change_log_detail.write(obj.__str__())

In [15]:
# close logs
change_log.close()
change_log_detail.close()